# The Remembering Problem

The "remembering problem" is a variant of the "adding problem" proposed by Schmidhuber and colleagues as an example of a sequential task that LSTM's are particularly well suited for: http://people.idsia.ch/~juergen/nipslstm/node4.html

### Mehrdad Yazdani
### November 12, 2018

Colab notebook online to play!!

https://colab.research.google.com/drive/13dNJ66vjiswqofcfcnecDeyY_DCeYYlq#scrollTo=qJMFpFxOiKja

![frequent words](https://blogs.sas.com/content/sastraining/files/2015/11/word_frequency.png)

Data source: http://norvig.com/google-books-common-words.txt


All methods will be compared using MSE on a held out test set. 

In [1]:
!pip install torch

In [2]:
import numpy as np
import pandas as pd
import random
import string
import collections


import matplotlib.pylab as plt
import seaborn as sns;
%matplotlib inline

In [3]:
import sys
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [4]:
# use CUDA or not
use_cuda = False
if use_cuda and torch.cuda.is_available():
  print("using cuda!")
  device = torch.device("cuda")
else:
  print("using CPU!")

using CPU!


## Data loading functions

We will define some helper functions to generate our datasets. `generate_sequence` will genrate a single sequence whereas `get_set` returns multiple sequences (so a *dataset* of sequences).



In [5]:
def generate_char_seq(seq_len = 10):
  return ''.join(random.choice(string.ascii_lowercase) for _ 
                 in range(seq_len))

def get_set(num_examples = 100, seq_len = 10):
  one_hot_encoded = {}
  for i, char in enumerate(list(string.ascii_lowercase)):
    one_hot_encoded[char] = i

  X_seqs = []
  num_repeats = []

  for _ in range(num_examples):
    seq_example = generate_char_seq(seq_len)
    X_seqs.append([one_hot_encoded[char] for char in list(seq_example)])
    num_repeats.append(collections.Counter(seq_example).most_common(1)[0][1])
    
  return np.array(X_seqs), np.array(num_repeats)  

Lets see `get_set` in action:

In [6]:
X_train, y_train = get_set(num_examples=100, seq_len = 3)
X_train[0,:]

array([21,  6,  9])

So for the input we have a 2D array that has shape "num examples" x "sequence length" 

Note that the datasets that `get_set` returns are Numpy arrays, but PyTorch recquires PyTorch tensors. We could of course convert these Numpy arrays to PyTorch arrays, and then do some booking with indices to keep track of going through different batches when doing batch updates on the network.

But that is tedious and PyTorch offers the Dataset class that we can inherit from to keep all this bookkeeping for us. Below we define the `SequenceDataset` generator class that will be used for all our data handilng for PyTorch. 

In [7]:
class SequenceDataset(Dataset):
  
  def __init__(self, num_examples, seq_len):
    self.num_examples = num_examples
    self.seq_len = seq_len
    
    X, y = get_set(num_examples=self.num_examples, seq_len = self.seq_len)
    self.X = torch.LongTensor(X)
    self.y = torch.from_numpy(y).float()
    if use_cuda and torch.cuda.is_available():
      self.X = self.X.to(device)
      self.y = self.y.to(device)
    
    
    
  def __getitem__(self, index):
    return self.X[index], self.y[index]
  
  def __len__(self):
    return self.num_examples

  

Lets create a training and test set with 100 examples for each and sequence lengths of 10. 

In [8]:
train_set = SequenceDataset(num_examples=100, seq_len = 10)
test_set = SequenceDataset(num_examples=100, seq_len = 10)



We can use PyTorch's `DataLoader` to specify the the batches of data to load for training. Note that each of the 100 example sequences are independent, so we also shuffle the order of the different sequences. 


In [9]:
batch_size = 32

train_loader = DataLoader(dataset = train_set,
                          batch_size=batch_size,
                          shuffle = True)

test_loader = DataLoader(dataset = test_set,
                         batch_size=batch_size,
                         shuffle = True)

## RNN

We will start solving the Remembering Problem with a simple RNN (the *Elman Network*). The network will update its internal hidden state for every element in the sequence until we reach the end. When we reach the end, we pass the final hidden state through a fully connected linear layer to predict the target. This type of architecture is sometimes called *many-to-one* since we are taking "many" elements (a sequence) to a single element (the target).

<center>
![Many to one](https://i.stack.imgur.com/QCnpU.jpg)
</center>

In [10]:
class RNNRemember(nn.Module):

    def __init__(self, hidden_size, embedding_size, input_size):    
        super(RNNRemember, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.embedding = nn.Embedding(input_size, self.embedding_size)
        self.rnn = nn.RNN(input_size=embedding_size,
                          hidden_size=self.hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size)
        #h_0 = Variable(torch.zeros(1, embedding_size, self.hidden_size))
        h_0 = Variable(torch.zeros(1, x.size(0), self.hidden_size))

        emb = self.embedding(x)
        # Propagate embedding through RNN
        # Input: (batch, seq_len, embedding_size)
        # h_0: (num_layers * num_directions, batch, hidden_size)
        _, h_f = self.rnn(emb, h_0)
        return self.fc(h_f).squeeze()

In [11]:
rnn_remember = RNNRemember(hidden_size = 50, embedding_size = 5, 
                           input_size = len(string.ascii_lowercase))

if use_cuda and torch.cuda.is_available():
    rnn_adder = rnn_adder.cuda(device)

In [12]:
# Set loss and optimizer function
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(rnn_remember.parameters(), lr=0.01)

In [13]:
%%time
num_epochs = 1000
for epoch in range(num_epochs):
  for i, (sequences, targets) in enumerate(train_loader):
    if use_cuda and torch.cuda.is_available():
      sequences = sequences.to(device)
      targets = targets.to(device)

    
    # forward pass
    outputs = rnn_remember(sequences)
    loss = criterion(outputs, targets)
    
    # update weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
  if (epoch+1)%100 == 0:
    print("loss is", loss.item())

loss is 0.0008117657853290439
loss is 0.0007380075985565782
loss is 1.7095851944759488e-05
loss is 0.0009870203211903572
loss is 3.625319777711411e-06
loss is 1.6261276414297754e-07
loss is 1.9569256437534932e-07
loss is 0.009279856458306313
loss is 0.0003926208592019975
loss is 1.5202015674731229e-05
CPU times: user 10.5 s, sys: 74 ms, total: 10.5 s
Wall time: 10.5 s


In [14]:
with torch.no_grad():
  outputs = rnn_remember(test_set.X)
  test_mse = torch.mean((outputs - test_set.y)**2)
print(test_mse.item())

0.4686804711818695


## LSTM

RNN's suffer from the vanishing gradient problem since creating the final hidden state is a result of updating the state through multiplications everytime a new element arrives in the sequence. LSTM's bypass this challenge by updating state additively. As a result, updaing gradients is much easier and longer memories can persist. Below is an `LSTMRemember` that is nearly identical to the `RNNRemember`



In [15]:
class LSTMRemember(nn.Module):

    def __init__(self, hidden_size, input_size, embedding_size):    
        super(LSTMRemember, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.embedding = nn.Embedding(input_size, self.embedding_size)
        self.lstm = nn.LSTM(input_size=self.embedding_size,
                          hidden_size=self.hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size)
        h_0 = Variable(torch.zeros(1, x.size(0), self.hidden_size))
        c_0 = Variable(torch.zeros(1, x.size(0), self.hidden_size))
        if use_cuda and torch.cuda.is_available():
          h_0 = h_0.to(device)
          c_0 = c_0.to(device)
          
                  
        emb = self.embedding(x)
        # Propagate input through LSTM
        # Input: (batch, seq_len, embedding_size)
        # h_0: (num_layers * num_directions, batch, hidden_size)
        _, (h_f, c_f) = self.lstm(emb, (h_0, c_0))
        return self.fc(h_f).squeeze()


In [16]:
lstm_remember = LSTMRemember(hidden_size = 50, embedding_size = 5, 
                           input_size = len(string.ascii_lowercase))
if use_cuda and torch.cuda.is_available():
    lstm_remember = lstm_remember.cuda(device)

In [17]:
# Set loss and optimizer function
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(lstm_remember.parameters(), lr=0.01)

In [18]:
%%time
num_epochs = 10000
for epoch in range(num_epochs):
  for i, (sequences, targets) in enumerate(train_loader):
    # forward pass
    outputs = lstm_remember(sequences)
    loss = criterion(outputs, targets)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
  if (epoch+1)%100 == 0:
    print("loss is", loss.item())

loss is 0.05242282152175903
loss is 0.004597179125994444
loss is 8.74446050147526e-06
loss is 5.7140850913128816e-06
loss is 0.0014133139047771692
loss is 2.0836645489907824e-05
loss is 0.003276211442425847
loss is 1.7728017382978578e-06
loss is 2.5956552462957916e-09
loss is 1.6632615285061547e-08
loss is 0.00013779960863757879
loss is 8.683727514835482e-09
loss is 1.732466969883717e-08
loss is 0.0011564528103917837
loss is 2.135296381311491e-05
loss is 7.596679643029347e-05
loss is 1.2502432582550682e-05
loss is 1.0631826626195107e-05
loss is 0.0006299148662947118
loss is 0.000237882457440719
loss is 1.7092472148760862e-07
loss is 1.939519734150963e-06
loss is 0.0010226396843791008
loss is 0.0004548224969767034
loss is 5.448233423521742e-05
loss is 7.408260671581957e-10
loss is 3.3146818623208674e-12
loss is 1.2825296380469808e-12
loss is 5.30038323631743e-06
loss is 0.00039897108217701316
loss is 2.3759685063851066e-05
loss is 8.414247076871106e-11
loss is 1.7763568394002505e-14
los

In [19]:
with torch.no_grad():
  outputs = lstm_remember(test_set.X)
  test_mse = torch.mean((outputs - test_set.y)**2)
print(test_mse.item())

0.44475048780441284


## ReLU RNN

The idea of the ReLU RNN is to initialize the hidden state of the RNN with the identity matrix and the bias with 0 and use the ReLU activation function. Below we demonstrate how such an RNN can be implemented. The results are not as good as the LSTM but certainly better than the traditional Elman Network.